In [ ]:
# import library ที่จำเป็น
import pandas as pd
import numpy as np

In [ ]:
# ค่า atomic property ที่ใช้งาน
#electronegativity polarizability van der Waals Volume
atom_property = {
'Br':[1.1723, 1.7330, 1.3835], 
 'F':[1.4567, 0.3165, 0.4100], 
 'Cr':[0.7200, 6.4444, 1.6283], 
 'Cu':[0.7403, 3.4659, 0.5120], 
 'N':[1.1631, 0.6250, 0.6949], 
 'H':[0.9439, 0.3790, 0.2989], 
 'Zn':[0.8095, 4.0341, 1.7084], 
 'O':[1.3307, 0.4557, 0.5120], 
 'P':[0.9159, 2.0625, 1.1814], 
 'V':[0.6392, 7.0455, 1.6283], 
 'S':[1.0768, 1.6477, 1.0882], 
 'Ba':[0.2487, 1.1252, 1.7941], 
 'C':[1.0000, 1.0000, 1.0000], 
 'I':[1.0117, 3.0398, 1.7280], 
 'Ni':[0.7283, 0.38636, 0.7643], 
 'Cl':[1.2655, 1.2386, 1.0347]
}

In [ ]:
# คำนวณค่า RDF (Radial Distribution Function)
def createFeatureRDF(data, structures, path3, _R):

    data_ = []

    for i, mof in enumerate(structures):

        if len(data[data['MOFname']==mof['name']]) == 0:
            continue
        
        # list ของ ธาตุใน structure
        el = mof['elements']
      
        d = {
            'MOFname':mof['name']
        }        


        # คำนวณตามแต่ละ R ที่กำหนด
        for R in _R: 
            
            sum1 = 0
            sum2 = 0
            sum3 = 0
            
            # ใช้วิธีการคำนวณแค่คู่เดียวและคูณสองแทน 
            
            for a in range(len(el)):
                # ตัวมันเอง rij = 0
                c = np.exp(-10*R**2)
                sum1 += atom_property[el[a]][0]*atom_property[el[a]][0]*c
                sum2 += atom_property[el[a]][1]*atom_property[el[a]][1]*c
                sum3 += atom_property[el[a]][2]*atom_property[el[a]][2]*c     
                
                for b in range(a+1, len(el)):
                    # หา ระยะห่างระหว่างอะตอม
                    rij = np.sqrt(np.sum((mof['coords'][a] - mof['coords'][b])**2))
                    
                    c = np.exp(-10*(rij-R)**2)
                    
                    sum1 += 2*atom_property[el[a]][0]*atom_property[el[b]][0]*c
                    sum2 += 2*atom_property[el[a]][1]*atom_property[el[b]][1]*c
                    sum3 += 2*atom_property[el[a]][2]*atom_property[el[b]][2]*c
            
            # บันทึกค่า R โดย normalize ด้วยจำนวนอะตอมรวม
            d[f'RDF-{R}-1'] = sum1/len(el)
            d[f'RDF-{R}-2'] = sum2/len(el)
            d[f'RDF-{R}-3'] = sum3/len(el)            

        if i%5000==0:
            print(i)
          
        data_.append(d)

    RDF_data = pd.DataFrame(data_4)
    
    RDF_data.to_csv(path3, index=False)
    
    print('FINISH')

    return structure_data

In [ ]:
# for trainset
# ตั้งชื่อให้สอดคล้องกับค่า R ที่ใช้คำนวณ
name = 'train_features_RDF_R_25_26_27_28_29_p5'

# อ่าน features ที่ extract จาก cif จาก cif_to_npy.ipynb
structures_train = np.load('train_structures.npy', allow_pickle=True)

# คำนวณค่า RDF จาก list ของ R 
result = createFeatureRDF(pd.read_csv('train.csv'), structures_train, name+'.csv', [25.5, 26.5, 27.5, 28.5, 29.5])
result

# ค่าที่คำนวณจริงจะแยกเป็นหลายไฟล์ ใช้หลายเครื่องในการช่วยคำนวณ

In [ ]:
# for testset

# ตั้งชื่อให้สอดคล้องกับค่า R ที่ใช้คำนวณ
name = 'test_features_RDF_R_25_26_27_28_29_p5'

# อ่าน features ที่ extract จาก cif จาก cif_to_npy.ipynb
structures_test = np.load('test_structures.npy', allow_pickle=True)

# คำนวณค่า RDF จาก list ของ R 
result = createFeatureRDF(pd.read_csv('test.csv'), structures_test, name+'.csv', [25.5, 26.5, 27.5, 28.5, 29.5])
result

# ค่าที่คำนวณจริงจะแยกเป็นหลายไฟล์ ใช้หลายเครื่องในการช่วยคำนวณ